In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project="ee-vishal15")

EEException: Project ee-vishal15 is not registered to use Earth Engine. Visit https://code.earthengine.google.com/register?project=ee-vishal15 to register your project. See https://developers.google.com/earth-engine/guides/access for more details.

#NDVI mapping

In [ ]:
import geemap
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="ee-vishal15")

# Define the region of interest (India)
india = ee.Geometry.Polygon([
    [[68.15, 37.12], [97.42, 37.12], [97.42, 6.75], [68.15, 6.75]]
])

# Load a Sentinel-2 image for a specific time range (for example, 2023)
image = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(india) \
    .filterDate('2023-01-01', '2023-12-31') \
    .median()  # Use the median of images in the period

# Compute NDVI (Normalized Difference Vegetation Index)
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Define visualization parameters for NDVI
ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Convert the India geometry to a FeatureCollection for displaying boundaries
india_boundary = ee.FeatureCollection([ee.Feature(india)])

# Initialize the map with center coordinates for India
Map = geemap.Map(center=[20.5937, 78.9629], zoom=5)

# Add the NDVI layer to the map
Map.addLayer(ndvi, ndvi_vis, 'NDVI')

# Add the India boundary layer for reference
Map.addLayer(india_boundary.style(color='black', width=2), {}, 'India Boundary')

# Display the map
Map


Map(center=[20.5937, 78.9629], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

##regionwise temperature assign

In [ ]:
import geemap
import ee

Map = geemap.Map()

temperature_data = ee.Image("MODIS/006/MOD11A1/2022_01_01")
temperature = temperature_data.select("LST_Day_1km")

min_temp = 250
max_temp = 3200
colors = [('blue'), ('green'), ('red')]

color_map = ee.List([min_temp, max_temp])

temperature_vis = temperature.visualize(min=min_temp, max=max_temp, palette=colors)

Map.addLayer(temperature_vis, {}, 'Temperature')

Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#Precipitation

In [ ]:
import geemap
import ee

ee.Authenticate()
ee.Initialize()

india_boundary = ee.Geometry.Polygon([
    [[68.15, 37.12], [97.42, 37.12], [97.42, 6.75], [68.15, 6.75]]
])

precipitation_dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterBounds(india_boundary) \
    .filterDate('2023-01-01', '2023-03-31')

total_precipitation = precipitation_dataset.sum()

precipitation_clipped = total_precipitation.clip(india_boundary)

precipitation_vis = {
    'min': 0,
    'max': 300,
    'palette': ['white', 'blue', 'green', 'yellow', 'red']
}

Map = geemap.Map(center=[20.5937, 78.9629], zoom=5)

Map.addLayer(precipitation_clipped, precipitation_vis, 'Total Precipitation (mm)')

Map.addLayer(ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
             .filter(ee.Filter.eq('country_na', 'India')),
             {'color': 'black'}, 'India Boundary')

Map


Map(center=[20.5937, 78.9629], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

#Co2 gases variation

In [ ]:
import geemap
import ee

Map = geemap.Map(center=[20.5937, 78.9629], zoom=5)

ee.Initialize()

india_boundary = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0') \
    .filter(ee.Filter.eq('ADM0_NAME', 'India'))

co2_collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CO') \
    .filterBounds(india_boundary) \
    .filterDate('2018-01-01', '2023-12-31')

co2_band = co2_collection.select('CO_column_number_density')

avg_co2 = co2_band.mean()

avg_co2_clipped = avg_co2.clip(india_boundary)

co2_vis = {
    'min': 0.02,
    'max': 0.05,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

Map.addLayer(avg_co2_clipped, co2_vis, 'CO Variation')

Map.addLayer(india_boundary.style(color='black', width=2), {}, 'India Boundary')

Map


Map(center=[20.5937, 78.9629], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

#merging various datasets into a single csv file

In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

dfs = []

for file_name in uploaded.keys():
    df = pd.read_csv(file_name)
    dfs.append(df)

df_combined = pd.concat(dfs, ignore_index=True)

df_combined.to_csv('combined_dataset.csv', index=False)

files.download('combined_dataset.csv')


TypeError: 'NoneType' object is not subscriptable